# COMP7015: Artificial Intelligence *(Semester 1, 2022/23)*

# Lab1: Solving problems using Search

In this lab session, we will learn how to solve problems using search algorithms. Specifically, we will learn the implementation of BFS, DFS, uniform-cost search and greedy search using the path-finding problem. The A* search will be left to you as an exercise.


**Instructor: Dr. Kejing Yin (Department of Computer Science, Hong Kong Baptist University)**

*This lab sheet is created by Dr. Kejing Yin and is licenced under MIT license.*

> MIT License
> 
> Copyright (c) 2022 Kejing Yin
> 
> Permission is hereby granted, free of charge, to any person obtaining a copy
> of this software and associated documentation files (the "Software"), to deal
> in the Software without restriction, including without limitation the rights
> to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
> copies of the Software, and to permit persons to whom the Software is
> furnished to do so, subject to the following conditions:
> 
> The above copyright notice and this permission notice shall be included in all
> copies or substantial portions of the Software.
> 
> THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
> IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
> FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
> AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
> LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
> OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
> SOFTWARE.

## 1. Finst-in, first-out (FIFO) approach and last-in, first-out (LIFO) approach

<div>
    <img src="figs/fifo_lifo.png"/>
</div>


**FIFO**: the last element inserted/recorded is consumed first.

**LIFO**: the oldest element is consumed first (e.g., washing a pile of plates in cafe).

In Python, the *easiest* way (not the *best* way though) is to use `list`. Let's see an example:

In [1]:
# Define am empty list named "frontiers"
frontiers = []
print(frontiers)

# We can simply append elements to the list
for x in ['A', 'B', 'C']:
    frontiers.append(x)
    print(frontiers)

[]
['A']
['A', 'B']
['A', 'B', 'C']


We use the `.pop(i)` method to retrieve the i-th position in the list and remove it from the list. We can use `.pop(0)` and `.pop(-1)` to get the first and the last elements in the list, respectively. For example:

In [2]:
print(frontiers.pop(0))  # get the first element in the list
print(frontiers)  # now the first element is removed from the list

A
['B', 'C']


In [3]:
print(frontiers.pop(-1))  # get the last element in the list
print(frontiers)  # now the last element is removed from the list

C
['B']


In our implementation of search algorithms, we use lists to store the frontier nodes and visited nodes.

* **Optional**: using `list` is not the best way because: 1) it can be quite inefficient in real-world applications, and 2) it only allows one producer and one consumer. A more decent way is to use the `queue` package provided by Python. See https://docs.python.org/3/library/queue.html for its documentation. If you are interested, try to use it to implement the search algorithms.

## 2. Pseudo-codes of search algorithms

All search algorithms can be summarized using the following pseudo codes.
</br>
</br>
<div>
    <img src="figs/pseudo_code.png" width=600px/>
</div>
The differences between different algorithms is how they choose the node to be expanded next, and when to test if the node is a goal state.

## 3. How to `expand` a node

For each search problem, we need to know all possible actions and the associated costs from a particular state and the resulting state of taking each action. In the path-finding problem, for each state (location in the map), we can simply store the locations that we can go to from the given location and the cost of taking each action. For example, given the following map:


<div>
    <img src="figs/map1.png" width="400px"/>
</div>

we can use a `dict` to represent this graph to be searched on.

In [4]:
graph = {
    'A': [('B', 1), ('D', 2)],  # from A, we can go to B and D
    'B': [('A', 1), ('C', 4), ('D', 2)],  # from B, we can go to A, C, and D
    'C': [('B', 4)],  # from C, we can go to B
    'D': [('A', 2), ('B', 2)]  # from D, we can got to A and B
}

In [5]:
# to retrieve the actions, we can use the current state to index this dict
print(graph['B'])  # print out the possible actions from the location B

[('A', 1), ('C', 4), ('D', 2)]


Later, we will see how to define a function to get possible actions from each state for other problems like 8-puzzle.

### *Try it out!*

Represent the following map in a `dict` named "romanian_graph".

<div>
    <img src="figs/romanian_graph.png" width="100%"/>
</div>


In [6]:
romanian_graph = {
    'Arad': [('Zerind', 75), ('Sibiu', 140), ('Timisoara', 118)],  # finish the remaining
    'Zerind': [('Oradea', 71), ('Arad', 75)],
    'Oradea': [('Zerind', 71), ('Sibiu', 151)],
    'Sibiu': [('Fagaras', 99), ('Rimnicu Vilcea', 80), ('Oradea', 151), ('Arad', 140)],
    'Timisoara': [('Lugoj', 111), ('Arad', 118)],
    'Lugoj': [('Mehadia', 70), ('Timisoara', 111)],
    'Mehadia': [('Drobeta', 75), ('Lugoj', 70)],
    'Drobeta': [('Craiova', 120), ('Mehadia', 75)],
    'Rimnicu Vilcea': [('Craiova', 146), ('Pitesti', 97), ('Sibiu', 80)],
    'Fagaras': [('Sibiu', 99), ('Bucharest', 211)],
    'Pitesti': [('Rimnicu Vilcea', 97), ('Craiova', 138), ('Bucharest', 101)],
    'Craiova': [('Rimnicu Vilcea', 146), ('Drobeta', 120), ('Pitesti', 138)],
    'Bucharest': [('Fagaras', 211), ('Pitesti', 101), ('Giurgiu', 90), ('Urzicenzi', 85)],
    'Giurgiu': [('Bucharest', 90)],
    'Urzicenzi': [('Bucharest', 85), ('Vaslui', 142), ('Hirsova', 98)],
    'Vaslui': [('Urzicenzi', 142), ('Iasi', 92)],
    'Iasi': [('Vaslui', 92), ('Neamt', 87)],
    'Neamt': [('Iasi', 87)],
    'Hirsova': [('Urzicenzi', 98), ('Eforie', 86)],
    'Eforie': [('Hirsova', 86)],
}

## 4. Breadth-first Search (BFS)

In [7]:
def bfs(graph_to_search, initial_state, goal_state, verbose=False):
    # this is a list of a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in this list represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost.
    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []

    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
        
        path = frontiers.pop(0)  # Get the first element in the list
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.
            
            # check if we reached the goal state or not
            if next_node == goal_state:
                goal_path = new_path[1:]
                goal_cost = new_path[0]
                return goal_path, goal_cost  # if yes, we can return this path and its cost
            else:
                frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)

    return None

In [8]:
graph = {
    'A': [('B', 1), ('D', 2)],
    'B': [('A', 1), ('C', 4), ('D', 2)],
    'C': [('B', 4)],
    'D': [('A', 2), ('B', 2)]
}

path, cost = bfs(graph, 'A', 'C')
print('The solution is:', path)
print('The cost is:', cost)

The solution is: ['A', 'B', 'C']
The cost is: 5


In [9]:
# set `verbose=True` to print out the frontiers and visited list in each iteration
path, cost = bfs(graph, 'A', 'C', verbose=True)

Frontiers (paths):
  - [0, 'A']
Visited: []


Frontiers (paths):
  - [1, 'A', 'B']
  - [2, 'A', 'D']
Visited: ['A']




### *Try it out!*

In [10]:
# Try it out!
# Find a path from Arad to Bucharest in the Romannian map using BFS

path, cost = bfs(romanian_graph, 'Arad', 'Bucharest', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
  - [146, 'Arad', 'Zerind', 'Oradea']
Visited: ['Arad', 'Zerind']


Frontiers (paths):
  - [118, 'Arad', 'Timisoara']
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Zerind', 'Sibiu']


Frontiers (paths):
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
Visited: ['Arad', 'Zerind', 'Sibiu', 'Timisoara']


Frontiers (paths):
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
  - [229, 'Arad', 'Timisoar

In [11]:
# Find a path from Arad to Craiova in the Romannian map using BFS

path, cost = bfs(romanian_graph, 'Arad', 'Craiova', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
  - [146, 'Arad', 'Zerind', 'Oradea']
Visited: ['Arad', 'Zerind']


Frontiers (paths):
  - [118, 'Arad', 'Timisoara']
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Zerind', 'Sibiu']


Frontiers (paths):
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
Visited: ['Arad', 'Zerind', 'Sibiu', 'Timisoara']


Frontiers (paths):
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
  - [229, 'Arad', 'Timisoar

In [12]:
# Find a path from Arad to Vaslui in the Romannian map using BFS

path, cost = bfs(romanian_graph, 'Arad', 'Vaslui', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
  - [146, 'Arad', 'Zerind', 'Oradea']
Visited: ['Arad', 'Zerind']


Frontiers (paths):
  - [118, 'Arad', 'Timisoara']
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Zerind', 'Sibiu']


Frontiers (paths):
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
Visited: ['Arad', 'Zerind', 'Sibiu', 'Timisoara']


Frontiers (paths):
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
  - [229, 'Arad', 'Timisoar

## 5. Graph Depth-first Search (DFS; graph)

When DFS is implemented in graph search, we record the visited nodes to avoid revisiting them. The implementation is very similar to BFS. The only difference is that we retrieve the last node from the frontiers instead of the first one.

### *Try it out!*
Copy the previous `bfs` function, rename it to `dfs` and modify it to perform a graph dfs. Test it with the Romannian map.

In [13]:
def dfs(graph_to_search, initial_state, goal_state, verbose=False):
    # this is a list of a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in this list represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost.
    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []

    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
        
        path = frontiers.pop(-1)  # Get the last element in the list
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.
            
            # check if we reached the goal state or not
            if next_node == goal_state:
                goal_path = new_path[1:]
                goal_cost = new_path[0]
                return goal_path, goal_cost  # if yes, we can return this path and its cost
            else:
                frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)

    return None

In [14]:
# Try it out!
# Find a path from Arad to Bucharest in the Romannian map using DFS

path, cost = dfs(romanian_graph, 'Arad', 'Bucharest', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
Visited: ['Arad', 'Timisoara']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [299, 'Arad', 'Timisoara', 'Lugoj', 'Mehadia']
Visited: ['Arad', 'Timisoara', 'Lugoj']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [374, 'Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta']
Visited: ['Arad', 'Timisoara', 'Lugoj', 'Mehadia']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [494, 'Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta', 'Craiova']
Visited: ['Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [640, 'Arad', 'Timisoara', 'Lugoj', 'Meh

In [15]:
# Find a path from Arad to Craiova in the Romannian map using DFS

path, cost = dfs(romanian_graph, 'Arad', 'Craiova', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
Visited: ['Arad', 'Timisoara']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [299, 'Arad', 'Timisoara', 'Lugoj', 'Mehadia']
Visited: ['Arad', 'Timisoara', 'Lugoj']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [374, 'Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta']
Visited: ['Arad', 'Timisoara', 'Lugoj', 'Mehadia']


The solution is: ['Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta', 'Craiova']
The cost is: 494


In [16]:
# Find a path from Arad to Vaslui in the Romannian map using DFS

path, cost = dfs(romanian_graph, 'Arad', 'Vaslui', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
Visited: ['Arad', 'Timisoara']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [299, 'Arad', 'Timisoara', 'Lugoj', 'Mehadia']
Visited: ['Arad', 'Timisoara', 'Lugoj']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [374, 'Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta']
Visited: ['Arad', 'Timisoara', 'Lugoj', 'Mehadia']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [494, 'Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta', 'Craiova']
Visited: ['Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [640, 'Arad', 'Timisoara', 'Lugoj', 'Meh

## 6. Uniform-cost Search

For uniform-cost search, there are two points that we need to pay attention to:
- First, we check if a node is a goal state **when we expand it**. Think about how to do this in the codes.
- Second, we select the node from frontiers with the **smallest path cost** from the initial state.

#### A hint: select the smallest path cost

In [17]:
# For example, we have the following list of frontiers:
frontiers = [
    [3, 'S', 'A'],
    [1, 'S', 'B'],
    [2, 'S', 'C']
]

# We could sort them first:
print('Before sorting:', frontiers)

frontiers = sorted(frontiers, key=lambda x: x[0])

print('After sorting: ', frontiers)


# Then, we can pop the first element.
print()
print(frontiers.pop(0))

Before sorting: [[3, 'S', 'A'], [1, 'S', 'B'], [2, 'S', 'C']]
After sorting:  [[1, 'S', 'B'], [2, 'S', 'C'], [3, 'S', 'A']]

[1, 'S', 'B']


### *Try it out!*
Make use of the above hint, copy the previous bfs function, rename it to `uniform_cost_search` and modify it to perform uniform cost search.

In [18]:
def uniform_cost_search(graph_to_search, initial_state, goal_state, verbose=False):
    # this is a list of a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in this list represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost.
    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []

    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
        
        frontiers = sorted(frontiers, key=lambda x: x[0])
        path = frontiers.pop(0)  # Get the first path in the queue
        node = path[-1]  # Get the last node in this path
        
        if node == goal_state:
            goal_path = path[1:]
            goal_cost = path[0]
            return goal_path, goal_cost
        
        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.
            
            frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)

    return None

In [19]:
# Try it out!
# Find a path from Arad to Bucharest in the Romannian map using uniform-cost search

path, cost = uniform_cost_search(romanian_graph, 'Arad', 'Bucharest', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [118, 'Arad', 'Timisoara']
  - [140, 'Arad', 'Sibiu']
  - [146, 'Arad', 'Zerind', 'Oradea']
Visited: ['Arad', 'Zerind']


Frontiers (paths):
  - [140, 'Arad', 'Sibiu']
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
Visited: ['Arad', 'Zerind', 'Timisoara']


Frontiers (paths):
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Zerind', 'Timisoara', 'Sibiu']


Frontiers (paths):
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Zerind', 'Timisoara', 'Sibiu', 'Oradea']


Front

In [20]:
# Find a path from Arad to Craiova in the Romannian map using uniform-cost search

path, cost = uniform_cost_search(romanian_graph, 'Arad', 'Craiova', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [118, 'Arad', 'Timisoara']
  - [140, 'Arad', 'Sibiu']
  - [146, 'Arad', 'Zerind', 'Oradea']
Visited: ['Arad', 'Zerind']


Frontiers (paths):
  - [140, 'Arad', 'Sibiu']
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
Visited: ['Arad', 'Zerind', 'Timisoara']


Frontiers (paths):
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Zerind', 'Timisoara', 'Sibiu']


Frontiers (paths):
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Zerind', 'Timisoara', 'Sibiu', 'Oradea']


Front

In [21]:
# Find a path from Arad to Vaslui in the Romannian map using uniform-cost search

path, cost = uniform_cost_search(romanian_graph, 'Arad', 'Vaslui', verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [118, 'Arad', 'Timisoara']
  - [140, 'Arad', 'Sibiu']
  - [146, 'Arad', 'Zerind', 'Oradea']
Visited: ['Arad', 'Zerind']


Frontiers (paths):
  - [140, 'Arad', 'Sibiu']
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
Visited: ['Arad', 'Zerind', 'Timisoara']


Frontiers (paths):
  - [146, 'Arad', 'Zerind', 'Oradea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Zerind', 'Timisoara', 'Sibiu']


Frontiers (paths):
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [229, 'Arad', 'Timisoara', 'Lugoj']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Zerind', 'Timisoara', 'Sibiu', 'Oradea']


Front

### Compare the path and cost found using the three methods, what is your observation? Explain the differences using what you learnt from the lecture.

## 7. Greedy Search
In greedy search, we need to define a **`heuristic function`** $h(n)$ that tells us the estimated cost of the cheapest path from a particular state to a goal state. In this path-finding problem, we make it simpler by using another `dict` to store the straight-line distance from each city to our goal, Bucharest.

In [22]:
def sld_to_bucharest(city):
    """This function returns the straight-line distance from the given city to Bucharest."""
    sld = {
        'Arad': 366,
        'Bucharest': 0,
        'Craiova': 160,
        'Drobeta': 242,
        'Eforie': 161,
        'Fagaras': 176,
        'Giurgiu': 77,
        'Hirsova': 151,
        'Iasi': 226,
        'Lugoj': 244,
        'Mehadia': 241,
        'Neamt': 234,
        'Oradea': 380,
        'Pitesti': 100,
        'Rimnicu Vilcea': 193,
        'Sibiu': 253,
        'Timisoara': 329,
        'Urzicenzi': 80,
        'Vaslui': 199,
        'Zerind': 374
    }
    return sld[city]

#### Hint: Find the element with the lowest $h(n)$ in a given list

In [23]:
# Suppose given the following cities
cities = ['Arad', 'Eforie', 'Oradea']

# We would like to pop the city that is closest to Bucharest from this list using the above sld_to_bucharest function.
# first, we can construct a list that contains the distance from each city to Bucharest.
dist = []
for city in cities:
    dist.append(sld_to_bucharest(city))

# now we have a list of distances from each city to Bucharest.
print(dist)

# then, we can find the index of the minimum value.
# min(dist) returns the minimum value in the dist list, 
# dist.index() finds the index of the given value
idx = dist.index(min(dist))
print('The index of the closest city is:', idx)

# then we can pop the city using the index found.
closest_city = cities.pop(idx)
print('The closest city is:', closest_city)
print('The distance is:', sld_to_bucharest(closest_city))
print('The remaining elements in `cities` are:', cities)

[366, 161, 380]
The index of the closest city is: 1
The closest city is: Eforie
The distance is: 161
The remaining elements in `cities` are: ['Arad', 'Oradea']


### *Try it out!*
The difference between greedy search and BFS is that we expand the node with *lowest* $h(n)$. Copy the bfs function, rename it to `greedy_search` and modify it with the help of the heuristic function we defined above to perform greedy search.

In [24]:
def greedy_search(graph_to_search, initial_state, goal_state, heuristics_function, verbose=False):
    # this is a list of a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in this list represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost.
    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []

    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
            
        # get the nodes in frontiers to be expanded
        frontier_heuristics = []
        for x in frontiers:
            frontier_heuristics.append(heuristics_function(x[-1]))
        idx_to_pop = frontier_heuristics.index(min(frontier_heuristics))
        
        path = frontiers.pop(idx_to_pop)
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.
            
            # check if we reached the goal state or not
            if next_node == goal_state:
                goal_path = new_path[1:]
                goal_cost = new_path[0]
                return goal_path, goal_cost  # if yes, we can return this path and its cost
            else:
                frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)

    return None

In [25]:
# Run this after you implement the algorithm, see if it gives you the correct answer.
# Find a path from Arad to Bucharest in the Romannian map using greedy search

path, cost = greedy_search(romanian_graph, 'Arad', 'Bucharest', heuristics_function=sld_to_bucharest, verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [118, 'Arad', 'Timisoara']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Sibiu']


The solution is: ['Arad', 'Sibiu', 'Fagaras', 'Bucharest']
The cost is: 450


## 8. A* Search
For A* search, we compute the sum of the cost of the path $g(n)$ and the heuristic function $h(n)$, and select the node with *lowest* $h(n)+g(n)$ to expand. Besides, similar to the uniform-cost search, we check the goal state when we **expand it**.  Copy the greedy search function, rename it to `a_star_search` and modify it to perform greedy search. You could make reference to the unifrom-cost search codes to check the goal state when expanding the node.

### *Try it out!*
Copy the greedy function, rename it to `a_star_search` and modify it to perform A* search.

In [26]:
def a_star_search(graph_to_search, initial_state, goal_state, heuristics_function, verbose=False):
    # this is a list of a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in this list represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost.
    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []

    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
            
        # get the nodes in frontiers to be expanded
        estimated_path_cost = []
        for x in frontiers:
            heuristic_value = heuristics_function(x[-1])
            path_cost = x[0]
            estimated_path_cost.append(heuristic_value+path_cost)
        idx_to_pop = estimated_path_cost.index(min(estimated_path_cost))
        
        path = frontiers.pop(idx_to_pop)
        node = path[-1]  # Get the last node in this path
        
        if node == goal_state:
            goal_path = path[1:]
            goal_cost = path[0]
            return goal_path, goal_cost
        
        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.

            frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)

    return None

In [27]:
# Try it out!
# Find a path from Arad to Bucharest in the Romannian map using A* search

path, cost = a_star_search(romanian_graph, 'Arad', 'Bucharest', heuristics_function=sld_to_bucharest, verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [118, 'Arad', 'Timisoara']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Sibiu']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [118, 'Arad', 'Timisoara']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [291, 'Arad', 'Sibiu', 'Oradea']
  - [366, 'Arad', 'Sibiu', 'Rimnicu Vilcea', 'Craiova']
  - [317, 'Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti']
Visited: ['Arad', 'Sibiu', 'Rimnicu Vilcea']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [118, 'Arad', 'Timisoara']
  - [291, 'Arad', 'Sibiu', 'Oradea']
  - [366, 'Arad', 'Sibiu', 'Rimnicu Vilcea', 'Craiova']
  - [317, 'Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti']
  - [450, 'Arad', 'Sibiu', 'Fagaras', 'Bucharest']
Visited: ['Ar

### What happens if we have a heuristic function that overestimates the cost?

### *Try it out!*
Define a heuristic function that returns the doubled straight-line distance from each city to Bucharest. Hint: you do not need to recode the dict again, just make use of the predefined `sld_to_bucharest` function.

In [28]:
def overestimated_heuristic_to_bucharest(city):
    """Define a heuristic function that gives you twice the straight-line distance from each city Bucharest."""
    sld = {
        'Arad': 366,
        'Bucharest': 0,
        'Craiova': 160,
        'Drobeta': 242,
        'Eforie': 161,
        'Fagaras': 176,
        'Giurgiu': 77,
        'Hirsova': 151,
        'Iasi': 226,
        'Lugoj': 244,
        'Mehadia': 241,
        'Neamt': 234,
        'Oradea': 380,
        'Pitesti': 100,
        'Rimnicu Vilcea': 193,
        'Sibiu': 253,
        'Timisoara': 329,
        'Urzicenzi': 80,
        'Vaslui': 199,
        'Zerind': 374
    }
    return 2*sld[city]

Perform A* search using this new heuristic function to find a way from Arad to Bucharest.

In [29]:
path, cost = a_star_search(romanian_graph, 'Arad', 'Bucharest', 
                           heuristics_function=overestimated_heuristic_to_bucharest, 
                           verbose=True)
print('The solution is:', path)
print('The cost is:', cost)

Frontiers (paths):
  - [0, 'Arad']
Visited: []


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [140, 'Arad', 'Sibiu']
  - [118, 'Arad', 'Timisoara']
Visited: ['Arad']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [118, 'Arad', 'Timisoara']
  - [239, 'Arad', 'Sibiu', 'Fagaras']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
Visited: ['Arad', 'Sibiu']


Frontiers (paths):
  - [75, 'Arad', 'Zerind']
  - [118, 'Arad', 'Timisoara']
  - [220, 'Arad', 'Sibiu', 'Rimnicu Vilcea']
  - [291, 'Arad', 'Sibiu', 'Oradea']
  - [450, 'Arad', 'Sibiu', 'Fagaras', 'Bucharest']
Visited: ['Arad', 'Sibiu', 'Fagaras']


The solution is: ['Arad', 'Sibiu', 'Fagaras', 'Bucharest']
The cost is: 450


### Compare the results of using the two different heuristic functions, what are your observations? Try to explain the differences using what we learnt from the lectures.